In [1]:
# Start orekit
import orekit
from math import radians, degrees
import matplotlib.pyplot as plt
orekit.initVM()

from orekit.pyhelpers import setup_orekit_curdir
setup_orekit_curdir()

In [14]:
# import necessary libraries
from org.orekit.frames import FramesFactory
from org.orekit.orbits import KeplerianOrbit
from org.orekit.orbits import PositionAngle
from org.orekit.propagation import SpacecraftState
from org.orekit.propagation.analytical import KeplerianPropagator
from org.orekit.time import AbsoluteDate
from org.orekit.time import TimeScalesFactory

# import the numerial propagator libraries
from org.orekit.propagation.numerical import NumericalPropagator
from org.hipparchus.ode.nonstiff import DormandPrince853Integrator
from org.orekit.orbits import OrbitType
from org.orekit.forces.gravity.potential import GravityFieldFactory
from org.orekit.forces.gravity import HolmesFeatherstoneAttractionModel
from org.orekit.forces.gravity import ThirdBodyAttraction
from org.orekit.utils import IERSConventions

from org.orekit.forces.radiation import SolarRadiationPressure
from org.orekit.forces.radiation import IsotropicRadiationSingleCoefficient

from org.orekit.bodies import CelestialBodyFactory
from org.orekit.propagation.events.handlers import EventHandler
from org.orekit.propagation.events import PositionAngleDetector
from org.orekit.orbits import PositionAngle

from org.orekit.propagation.events import EventDetector
from org.orekit.propagation.events.handlers import EventHandler
from org.orekit.propagation.events import PositionAngleDetector
from org.orekit.propagation.events import AbstractDetector
from org.orekit.orbits import OrbitType
from org.orekit.orbits import PositionAngle

from org.orekit.forces.maneuvers import ImpulseManeuver
from org.orekit.propagation.events import ApsideDetector 

from org.orekit.propagation.sampling import OrekitFixedStepHandler
from org.orekit.python import PythonOrekitFixedStepHandler, PythonEventHandler


In [3]:
# Set the initial State of the spacecraft
a = 24396159.0    # semi major axis (m)
e = 0.720  # eccentricity
i = radians(10.0) # inclination
omega = radians(50.0) # perigee argument
raan = radians(150.0) #right ascension of ascending node
lM = 0.0 # mean anomaly

# Set inertial frame
inertialFrame = FramesFactory.getEME2000()

# Initial date in UTC time scale
utc = TimeScalesFactory.getUTC()
initialDate = AbsoluteDate(2016, 1, 1, 5, 30, 00.00, utc)  # year, month, day, hours,mintues, seconds

# Setup orbit propagator
#gravitation coefficient
mu = 3.986004415e+14

# Orbit construction as Keplerian
initialOrbit = KeplerianOrbit(a, e, i, omega, raan, lM,
                              PositionAngle.MEAN, inertialFrame,
                              initialDate, mu)

initialstate = SpacecraftState(initialOrbit)

In [4]:
def simple_keplarian(initialOrbit, initialDate):
    """
    :param initialOrbit: initial Keplarian orbit and central body
    :param initialDate: intial start date
    :return: plot xy orbit
    """

    # Simple extrapolation with Keplerian motion
    kepler = KeplerianPropagator(initialOrbit)
    # Set the propagator to slave mode (could be omitted as it is the default mode)
    kepler.setSlaveMode()
    # Setup propagation time
    # Overall duration in seconds for extrapolation
    duration = 24 * 60.0 ** 2
    # Stop date
    finalDate = AbsoluteDate(initialDate, duration, utc)
    # Step duration in seconds
    stepT = 30.0
    # Perform propagation
    # Extrapolation loop
    cpt = 1.0
    extrapDate = initialDate
    px, py = [], []
    while extrapDate.compareTo(finalDate) <= 0:
        currentState = kepler.propagate(extrapDate)
        print('step {}: time {} {}\n'.format(cpt, currentState.getDate(), currentState.getOrbit()))
        coord = currentState.getPVCoordinates().getPosition()
        px.append(coord.getX())
        py.append(coord.getY())
        # P[:,cpt]=[coord.getX coord.getY coord.getZ]
        extrapDate = AbsoluteDate(extrapDate, stepT, utc)
        cpt += 1
    plt.plot(px, py)
    plt.show()

In [5]:
initialDate.shiftedBy(1000.0)

<AbsoluteDate: 2016-01-01T05:46:40.000>

In [6]:
# use a numerical propogator and integrator
min_step = 0.001
max_step = 1000.0
position_tolerance = 10.0

propagation_type = OrbitType.KEPLERIAN
tolerances = NumericalPropagator.tolerances(position_tolerance, initialOrbit,
                                            propagation_type)

integrator = DormandPrince853Integrator(min_step, max_step, 1e-4, 1e-5)

propagator = NumericalPropagator(integrator)
propagator.setOrbitType(propagation_type)

In [7]:
class TutorialStepHandler(PythonOrekitFixedStepHandler):

    def init(self,s0, t):
        print('Orbial Elements')
    
    def handleStep(self, currentState, isLast):
        print('State')
        o = OrbitType.KEPLERIAN.convertType(currentState.getOrbit())
        print(o.getA()," ", o.getE(),' ta:' ,degrees(o.getLv()))
#         print("%s %:12.3f %:10.8f %:106f %:10.6f %:10.6f %:10.6f".format(currentState.getDate(), o.getA(), o.getE(),
#                                                                       o.getI(),o.getPerigeeArgument(),
#                                                                       o.getRightAscensionOfAscendingNode(),
#                                                                       o.getTrueAnomaly()))
        if isLast:
            print('this was the last step ')

In [8]:
class AnomalyHandler(PythonEventHandler):

    def init():
        pass
    
    def eventOccured(self, sc_state, detector, increasing):
        if increasing:
            print('Reached target TA : ', sc_state.getDate())
        return EventHandler.Action.CONTINUE

    def resetState(self, detector, oldstate):
        return oldstate

In [9]:
# force model gravity field
provider = GravityFieldFactory.getNormalizedProvider(10, 10)
holmesFeatherstone = HolmesFeatherstoneAttractionModel(FramesFactory.getITRF(IERSConventions.IERS_2010, True), provider)

# Add Earth, Moon, and Sun body gravity field
propagator.addForceModel(holmesFeatherstone)
propagator.addForceModel(ThirdBodyAttraction(CelestialBodyFactory.getSun()))
propagator.addForceModel(ThirdBodyAttraction(CelestialBodyFactory.getMoon()))

# SRP
ssc = IsotropicRadiationSingleCoefficient(100.0, 0.8)   # Spacecraft surface area (m^2), C_r absorbtion 
spr = SolarRadiationPressure(CelestialBodyFactory.getSun(), a, ssc)   # sun, semi-major Earth, spacecraft sensitivity

# propagator.addForceModel(spr)


In [10]:
print(PositionAngle.TRUE)

TRUE


In [11]:
    max_check = 100.0
    threshold = 1.0
    true_anomaly = radians(253.0)
    detect_ta_event = PositionAngleDetector(OrbitType.KEPLERIAN, PositionAngle.TRUE, true_anomaly).withHandler(AnomalyHandler())
#     detect_ta_event = detect_ta_event.withHandler(AnomalyHandler().of_(PositionAngleDetector()))
    propagator.addEventDetector(detect_ta_event)

In [12]:
propagator.setMasterMode(60.0, TutorialStepHandler())

propagator.setInitialState(initialstate)

finalState = propagator.propagate(initialDate.shiftedBy(1000.0))    # TIme shift in seconds

o = OrbitType.KEPLERIAN.convertType(finalState.getOrbit())

print('Final State: date: {}\na: {} \ne: {} \ni:{} \ntheta {}'.format(
       finalState.getDate(), o.getA(), o.getE(), degrees(o.getI()), degrees(o.getLv())))

print("done")

Orbial Elements
State
24396159.0   0.72  ta: 200.0
State
24395024.943484217   0.7199861495571943  ta: 205.03631633600043
State
24393239.01404412   0.7199632146976402  ta: 210.04033674189114
State
24390878.57747052   0.7199322308258499  ta: 214.98115133236405
State
24388030.164556943   0.7198943938820179  ta: 219.83043405989204
State
24384797.33770412   0.7198511358662764  ta: 224.56340277251857
State
24381298.27972513   0.7198040742136987  ta: 229.15944191804735
State
24377654.93920867   0.7197548618945949  ta: 233.60239598609547
State
24373977.547326162   0.719704988762275  ta: 237.8805491702975
State
24370355.75871288   0.7196556700554514  ta: 241.9863386324296
State
24366855.338449318   0.7196078061384754  ta: 245.91591248401176
State
24363519.68628024   0.7195620036732762  ta: 249.66858054179596
State
24360374.09970637   0.7195186308120338  ta: 253.2462384717467
State
24357430.06718809   0.7194778725937937  ta: 256.6528110050526
State
24354689.32227059   0.7194397830934615  ta: 259

In [72]:
duration = 24 * 60.0 ** 2
# Stop date
finalDate = AbsoluteDate(initialDate, duration, utc)
# Step duration in seconds
stepT = 30.0
# Perform propagation
# Extrapolation loop
cpt = 1.0
px = []
py = []
extrapDate = initialDate
while extrapDate.compareTo(finalDate) <= 0:
    currentState = propagator.propagate(extrapDate)
    # print('step {}: time {} {}\n'.format(cpt, currentState.getDate(), currentState.getOrbit()))
    coord = currentState.getPVCoordinates().getPosition()
    px.append(coord.getX())
    py.append(coord.getY())
    # P[:,cpt]=[coord.getX coord.getY coord.getZ]
    extrapDate = AbsoluteDate(extrapDate, stepT, utc)
    
plt.plot(px,py)
plt.show()

Orbial Elements
Step handler
(24086154.609394196, ' ', 0.7156957936460763, ' ta:', 290.3867919625237)
Step handler
(24090873.88653376, ' ', 0.7157646407217769, ' ta:', 288.6434366612754)
Step handler
(24095989.383335408, ' ', 0.7158391971353858, ' ta:', 286.82255383591564)
Step handler
(24101537.87193505, ' ', 0.7159199755368835, ' ta:', 284.9189751400686)
Step handler
(24107559.17171778, ' ', 0.7160075264285019, ' ta:', 282.9271699281565)
Step handler
(24114096.221837386, ' ', 0.7161024375240996, ' ta:', 280.8412362136636)
Step handler
(24121195.109025925, ' ', 0.7162053320276003, ' ta:', 278.654898832908)
Step handler
(24128905.018341076, ' ', 0.716316865387917, ' ta:', 276.36151788435524)
Step handler
(24137278.06992692, ' ', 0.7164377200463852, ' ta:', 273.95411043145526)
Step handler
(24146367.889520958, ' ', 0.7165685832042621, ' ta:', 271.4254018702077)
Step handler
(24156227.596766703, ' ', 0.7167101163412927, ' ta:', 268.7679013422463)
Step handler
(24166926.366242796, ' ', 0.

JavaError: <super: <class 'JavaError'>, <JavaError object>>
    Java stacktrace:
java.lang.RuntimeException: AttributeError
	at org.orekit.python.PythonEventHandler.eventOccurred(Native Method)
	at org.orekit.propagation.events.AbstractDetector.eventOccurred(AbstractDetector.java:158)
	at org.orekit.propagation.integration.AbstractIntegratedPropagator$AdaptedEventDetector.eventOccurred(AbstractIntegratedPropagator.java:825)
	at org.hipparchus.ode.events.EventState.doEvent(EventState.java:469)
	at org.hipparchus.ode.AbstractIntegrator.acceptStep(AbstractIntegrator.java:350)
	at org.hipparchus.ode.nonstiff.EmbeddedRungeKuttaIntegrator.integrate(EmbeddedRungeKuttaIntegrator.java:290)
	at org.orekit.propagation.integration.AbstractIntegratedPropagator.propagate(AbstractIntegratedPropagator.java:469)
	at org.orekit.propagation.integration.AbstractIntegratedPropagator.propagate(AbstractIntegratedPropagator.java:414)
	at org.orekit.propagation.integration.AbstractIntegratedPropagator.propagat